In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
start_time = time.time()
import random
from sklearn.preprocessing import RobustScaler
from functools import reduce
from keras import backend as K
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
np.random.seed(42)
random.seed(42)

Windows: 1,2,3,6,12,22,45,90

In [ ]:
numberOfWindows = 1
Restriction = '90'
ep = 125
numberOfFeatures = 1

In [ ]:
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

los_5

In [ ]:
FeatureSetB2 = pd.DataFrame()
FeatureSetB3 = pd.DataFrame()
FeatureSetB4 = pd.DataFrame()

In [ ]:
FeatureSet1 = pd.read_csv('DI/DATA_Base.csv') #Split
FeatureSet2 = pd.read_csv(f'DI/DATA_{numberOfWindows}.csv')#Train

In [ ]:
FeatureSetB2 = pd.DataFrame()
FeatureSetB1 = pd.DataFrame()
FeatureSetB4 = pd.DataFrame()

In [ ]:
FeatureSetB2['SubjectId'] = FeatureSet2['subject_id']
FeatureSetB2['BT'] = FeatureSet2['sleep']

In [ ]:
FeatureSet1['SubjectId'] = FeatureSet1['subject_id']
FeatureSetB1 = FeatureSet1.drop(['sleep','subject_id'], axis=1)

In [ ]:

FeatureSetB1.drop("ssAgg",axis=1,inplace = True)
FeatureSetB1.drop("usAgg",axis=1,inplace = True)
FeatureSetB1.drop("isAgg",axis=1,inplace = True)
FeatureSetB1.drop("dsAgg",axis=1,inplace = True)
FeatureSetB1.drop("disAgg",axis=1,inplace = True)


FeatureSetB1 = FeatureSetB1.fillna(0)
FeatureSetB2 = FeatureSetB2.fillna(0)

FeatureMortality2 = pd.read_csv('DI/Sleep_Tbl.csv') #Labels
FeatureMortality = pd.DataFrame()
FeatureMortality['SubjectId'] = FeatureMortality2['ClientId']
FeatureMortality['Death'] = FeatureMortality2['Flags']

FeatureSetB1.sort_values(by=['SubjectId'], ascending=True,inplace=True)
FeatureSetB2.sort_values(by=['SubjectId'], ascending=True,inplace=True)

FeatureMortality.sort_values(by=['SubjectId'], ascending=True,inplace=True)

#Chronic is 1        #Episodic is 2       #Transitional is 0

In [ ]:
FeatureMortality.Death[FeatureMortality.Death == 2] = 0      

In [ ]:
SubjectId = pd.DataFrame()
SubjectId = FeatureSetB1.SubjectId
SubjectId.drop_duplicates(keep = 'first', inplace = True)
SubjectId.reset_index(drop=True,inplace=True)
NumSubjects = len(SubjectId)
X = FeatureMortality['SubjectId'].equals(SubjectId)

In [ ]:
X = FeatureMortality['SubjectId'].isin(SubjectId)
FeatureMortality = FeatureMortality[X]
FeatureMortality.sort_values(by=['SubjectId'], ascending=True,inplace=True) 

In [ ]:
Labels = pd.DataFrame()
Labels['SubjectId'] = FeatureMortality['SubjectId']
Labels['Expired'] = FeatureMortality['Death']
y = Labels

In [ ]:
y['Expired'].value_counts()

In [ ]:
SubjectId = pd.DataFrame()
SubjectId = y.SubjectId
SubjectId.drop_duplicates(keep = 'first', inplace = True)
SubjectId.reset_index(drop=True,inplace=True)
X = FeatureSetB1['SubjectId'].isin(SubjectId)
FeatureSetB1 = FeatureSetB1[X]

In [ ]:
X = FeatureSetB1
X_1 = [X,y]

X_2 = reduce(lambda  left,right: pd.merge(left,right,on=['SubjectId'],how='outer'), X_1)
X_2.sort_values(by=['SubjectId'], ascending=True,inplace=True)

In [ ]:
X = FeatureSetB1

X_train1, X_test = train_test_split(X_2, test_size=0.2, random_state=42, stratify=X_2[['Expired']], shuffle=True)#,'AIDS','MetaStatic','Hematologic'#,'GCSEyes']]

X_train, X_val = train_test_split(X_train1, test_size=0.2, random_state=42, stratify=X_train1[['Expired']], shuffle=True)#, shuffle=True)#,'GCSEyes']],

In [ ]:
X_test['Expired'].value_counts()

In [ ]:
X_val['Expired'].value_counts()

In [ ]:
X_train['Expired'].value_counts()

In [ ]:
Subject_Test = pd.DataFrame()
Subject_Test = X_test.SubjectId
Subject_Test.drop_duplicates(keep = 'first', inplace = True)

Subject_Train = pd.DataFrame()
Subject_Train = X_train.SubjectId
Subject_Train.drop_duplicates(keep = 'first', inplace = True)

Subject_Val = pd.DataFrame()
Subject_Val = X_val.SubjectId
Subject_Val.drop_duplicates(keep = 'first', inplace = True)

In [ ]:
X = FeatureMortality['SubjectId'].isin(Subject_Train)
y_train = FeatureMortality[X]

X = FeatureMortality['SubjectId'].isin(Subject_Test)
y_test = FeatureMortality[X]

X = FeatureMortality['SubjectId'].isin(Subject_Val)
y_val = FeatureMortality[X]

y_train = y_train.drop(['SubjectId'], axis=1)
y_test = y_test.drop(['SubjectId'], axis=1)
y_val = y_val.drop(['SubjectId'], axis=1)

In [ ]:
X = FeatureSetB1['SubjectId'].isin(Subject_Train)
X_train = FeatureSetB1[X]
X2 = FeatureSetB2['SubjectId'].isin(Subject_Train)
X_train2 = FeatureSetB2[X2]

X = FeatureSetB1['SubjectId'].isin(Subject_Test)
X_test = FeatureSetB1[X]
X2 = FeatureSetB2['SubjectId'].isin(Subject_Test)
X_test2 = FeatureSetB2[X2]

X = FeatureSetB1['SubjectId'].isin(Subject_Val)
X_val = FeatureSetB1[X]
X2 = FeatureSetB2['SubjectId'].isin(Subject_Val)
X_val2 = FeatureSetB2[X2]

X_train.sort_values(by=['SubjectId'], ascending=True,inplace=True)
X_train2.sort_values(by=['SubjectId'], ascending=True,inplace=True)

X_test.sort_values(by=['SubjectId'], ascending=True,inplace=True)
X_test2.sort_values(by=['SubjectId'], ascending=True,inplace=True)

X_val.sort_values(by=['SubjectId'], ascending=True,inplace=True)
X_val2.sort_values(by=['SubjectId'], ascending=True,inplace=True)

In [ ]:
X_train = X_train.drop(['SubjectId','index_id','Unnamed: 0'], axis=1)
X_test = X_test.drop(['SubjectId','index_id','Unnamed: 0'], axis=1)
X_val = X_val.drop(['SubjectId','index_id','Unnamed: 0'], axis=1)

In [ ]:
X_train2 = X_train2.drop(['SubjectId'], axis=1)
X_test2 = X_test2.drop(['SubjectId'], axis=1)
X_val2 = X_val2.drop(['SubjectId'], axis=1)

In [ ]:
Train_shape = X_train.shape
Test_shape = X_test.shape
Val_shape = X_val.shape

In [ ]:
X_train = X_train.to_numpy()
X_train2 = X_train2.to_numpy()

X_test = X_test.to_numpy()
X_test2 = X_test2.to_numpy()

X_val = X_val.to_numpy()
X_val2 = X_val2.to_numpy()

In [ ]:
X_train2 = X_train2.reshape(int(Train_shape[0]),numberOfWindows*numberOfFeatures)
X_test2 = X_test2.reshape(int(Test_shape[0]),numberOfWindows*numberOfFeatures)
X_val2 = X_val2.reshape(int(Val_shape[0]),numberOfWindows*numberOfFeatures)

In [ ]:
X_train4 = np.concatenate((X_train2, X_train), axis=1)

X_test4 = np.concatenate((X_test2, X_test), axis=1)

X_val4 = np.concatenate((X_val2, X_val), axis=1)

In [ ]:
y_train = y_train['Death']
y_train = y_train.to_numpy()
y_train = y_train.flatten()

y_test = y_test['Death']
y_test = y_test.to_numpy()
y_test = y_test.flatten()

y_val = y_val['Death']
y_val = y_val.to_numpy()
y_val = y_val.flatten()

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Testing LR

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import sys
mod=sys.modules[__name__]

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 45, mode='min',restore_best_weights=True)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight

acc_score = []
re_score = []
pre_score = []
history_1 = []
f_score = []
acc_macro_score = [] 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn import preprocessing

In [ ]:
numberOfFeatures2 = X_test4.shape[1]

In [ ]:
scaler1 = StandardScaler().fit(X_train4)
scaler2 = StandardScaler().fit(X_test4)
scaler3 = StandardScaler().fit(X_val4)

X_train4 = scaler1.transform(X_train4)
X_test4 = scaler2.transform(X_test4)
X_val4 = scaler3.transform(X_val4)



for x in range(40):

    model = Sequential()
    model.add(tf.keras.Input(shape=(numberOfFeatures2,))) #(numberOfFeatures,)

    model.add(Dense(128))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    model.add(Dense(64))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    model.add(Dense(1, activation='sigmoid'))
    model.summary()


    opt = tf.keras.optimizers.legacy.Adam()#, decay=1e-3) #If validation is not following training in trend then the learning rate might be too high
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])#focal_loss_custom(alpha=0.2, gamma=2) #'binary_crossentropy'#focal_loss_custom(alpha=0.25, gamma=2)


    class_weights = class_weight.compute_class_weight(class_weight ='balanced',classes = np.unique(y_train),y = y_train)
    class_weights = dict(zip(np.unique(y_train), class_weights))

    history = model.fit(X_train4,y_train, epochs = ep,batch_size=256,validation_data=(X_val4, y_val), callbacks=[stop_early],shuffle=True,class_weight=class_weights)#,use_multiprocessing=True
    history_1.append(history)

    y_preds = model.predict(X_test4)
    y_preds = y_preds.flatten()
    y_pred = list(map(lambda x: 0 if x<0.5 else 1, y_preds))

    y_actu = pd.Series(y_test)
    y_pred = pd.Series(y_pred)
        
    sensitivity = recall_score(y_actu, y_pred,average='macro')#,pos_label = 1, average='binary')
    precision = precision_score(y_actu, y_pred,average='macro')#,pos_label = 1, average='binary')
    f1_value = f1_score(y_actu, y_pred,average='macro')#,pos_label = 1, average='binary')
    accuracy = accuracy_score(y_actu, y_pred)

    acc_score.append(accuracy)
    re_score.append(sensitivity)
    pre_score.append(precision)
    f_score.append(f1_value)
    
    matrix = matrix2 = confusion_matrix(y_actu, y_pred)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = matrix2)
    matrix.diagonal()/matrix.sum(axis=1)
    Accuracies = matrix.diagonal()/matrix.sum(axis=1)
    Less10D = Accuracies[0]
    Greater10D = Accuracies[1]
    Accuracies = matrix.diagonal()/matrix.sum(axis=1)
    accuracy_macro = (Greater10D+Less10D)/2
    acc_macro_score.append(accuracy_macro)

In [ ]:
k = 40
avg_acc_score = sum(acc_score)/k
avg_recall_score = sum(re_score)/k
avg_precision_score = sum(pre_score)/k
avg_f1_score = sum(f_score)/k
avg_acc_macro_score = sum(acc_macro_score)/k

accuracy_macro = avg_acc_macro_score
sensitivity = avg_recall_score
precision = avg_precision_score
accuracy = avg_acc_score
f1_score = avg_f1_score

In [ ]:
import math
def calculate_standard_deviation(numbers):
    n = len(numbers)
    mean = sum(numbers) / n
    variance = sum((x - mean) ** 2 for x in numbers) / n
    std_dev = math.sqrt(variance)
    return std_dev

std_acc = calculate_standard_deviation(acc_score)
std_acc_macro = calculate_standard_deviation(acc_macro_score)
std_recall = calculate_standard_deviation(re_score)
std_precision = calculate_standard_deviation(pre_score)
std_f1_score = calculate_standard_deviation(f_score)

In [ ]:
sensitivity_min = min(re_score)
sensitivity_max = max(re_score)

precision_min = min(pre_score)
precision_max = max(pre_score)

accuracy_min = min(acc_score)
accuracy_max = max(acc_score)

accuracy_macro_min = min(acc_macro_score)
accuracy_macro_max = max(acc_macro_score)

f1_score_min = min(f_score)
f1_score_max = max(f_score)

In [ ]:
print('Avg accuracy : {}'.format(avg_acc_score))
print('Avg Reccall : {}'.format(avg_recall_score))
print('Avg Precision : {}'.format(avg_precision_score))
print('Avg F1_score : {}'.format(avg_f1_score))
print('Avg accuracy_macro : {}'.format(avg_acc_macro_score))

In [ ]:
for history in history_1:
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='upper left')
    plt.show()

In [ ]:
for history in history_1:
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Validation Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='upper left')
    plt.show()

In [ ]:
import csv
import os.path
from datetime import datetime
end_time = time.time()
runtime = end_time - start_time

csv_columns = ['model-type','precision','sensitivity','f1-score','accuracy','accuracy_macro','NumberOfWindows','Epochs','Run_Time','Restriction','Acc_Lesser','Acc_Greater']
dict_data = [{'model-type':'ANN', 'precision': precision,'sensitivity': sensitivity,'f1-score': f1_score,'accuracy': accuracy,'accuracy_macro': accuracy_macro,'NumberOfWindows':numberOfWindows,"Epochs":ep,'Run_Time':runtime,'Restriction' : Restriction,'Acc_Lesser':Less10D,'Acc_Greater':Greater10D}]
metric_file = "Results/Chronic_ANN_Mid_Multi_1_Norm.csv"

file_exists = os.path.isfile(metric_file)
try:
    with open(metric_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        if not file_exists:
            writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")